In [1]:
import requests


In [2]:
input_path ="diffusion_input/"
pretrained_model_path ="diffusion_model_pretrained/"
output_path ="diffusion_output/"


In [3]:
import pandas as pd

dataset1= pd.read_csv(input_path+"dataset1.csv")
dataset2= pd.read_csv(input_path+"dataset2.csv")
dataset3= pd.read_csv(input_path+"dataset3.csv")
dataset4= pd.read_csv(input_path+"dataset3.csv")
dataset2_hindi= pd.read_csv(input_path+"dataset2_hindi.csv")
dataset2_hindi.rename(columns={"Prompt (Hindi)":"Prompt"}, inplace=True)

print(dataset1.shape,
      dataset2.shape,
      dataset3.shape,
      dataset4.shape,
      dataset2_hindi.shape,
      )



(49, 3) (100, 2) (137, 2) (137, 2) (70, 2)


In [ ]:
for i in range(len(dataset2)):
    prompt = dataset2["Prompt"][i]
    response = requests.post(
         f"https://api.stability.ai/v2beta/stable-image/generate/ultra",
         
         headers=
         {"authorization": 
          f"sk-COW9wUFOcp8tIBRoA5wcnqO4B8MpTPb4Mqb9VpcxaPmYCttg",
          "accept": 
          "image/*"},
          files={"none": ''},
          data={"prompt": 
                prompt,
                "output_format": "webp",},)
    if response.status_code == 200:
         with open(output_path+"dataset2"+"/SD_api/"+f"{prompt}.webp", 'wb') as file:
               file.write(response.content)
    else:
        raise Exception(str(response.json()))

Exception: {'errors': ['You lack sufficient credits to make this request.  Please purchase more credits at https://platform.stability.ai/account/credits and try again.'], 'id': '8885fdc3176cb2eee55c644d0ba1c6a1', 'name': 'payment_required'}

In [7]:
import base64
import os
import requests

engine_id = "stable-diffusion-xl-1024-v1-0"
api_host = os.getenv('API_HOST', 'https://api.stability.ai')
api_key = "sk-PliGGuj8zANVxdvQT0tQm1UpYbaKhCLq57gemyx71LszpLq7"

if api_key is None:
    raise Exception("Missing Stability API key.")


for i in range(len(dataset2)):
    prompt = dataset2["Prompt"][i]
    response = requests.post(
        f"{api_host}/v1/generation/{engine_id}/text-to-image",
        headers={
            "Content-Type": "application/json",
            "Accept": "application/json",
            "Authorization": f"Bearer {api_key}"
        },
        json={
            "text_prompts": [
                {
                    "text": prompt
                }
            ],
            "cfg_scale": 7,
            "height": 1024,
            "width": 1024,
            "samples": 1,
            "steps": 30,
        },
    )

    if response.status_code != 200:
        raise Exception("Non-200 response: " + str(response.text))

    data = response.json()

    for i, image in enumerate(data["artifacts"]):
        with open(output_path+"dataset2"+"/SD_api/"+f"{prompt}.webp", "wb") as f:
            f.write(base64.b64decode(image["base64"]))


Exception: Non-200 response: {"id":"0df4c4fd711b9d448a0fe42ae5e3023b","message":"Your organization does not have enough balance to request this action (need $0.009, have $0.007 in active grants, $0 in balance).","name":"insufficient_balance"}
